In [1]:
"""
~/datasets/...


phage_fragmented/
x4
    - Dataset-1_temperate.fasta

        => >Temp_gi|149|ref|NC_013055| Burkholderia phage KS9 linear:0-12169:1
            => >Temp
    idem pour viru

    - Dataset-1_virulent.fasta
    - Dataset-2_temperate.fasta
    - Dataset-2_virulent.fasta
    
refseq_simulated_metagenome/
    - host_chr_pvog_fragments.fasta
       => BACT


>Viru
ACTG

>Temp
ACTG

>Bact
ACTG

"""

'\n~/datasets/...\n\n\nphage_fragmented/\nx4\n    - Dataset-1_temperate.fasta\n\n        => >Temp_gi|149|ref|NC_013055| Burkholderia phage KS9 linear:0-12169:1\n            => >Temp\n    idem pour viru\n\n    - Dataset-1_virulent.fasta\n    - Dataset-2_temperate.fasta\n    - Dataset-2_virulent.fasta\n    \nrefseq_simulated_metagenome/\n    - host_chr_pvog_fragments.fasta\n       => BACT\n\n\n>Viru\nACTG\n\n>Temp\nACTG\n\n>Bact\nACTG\n\n'

In [2]:
import os

# Chemins vers les dossiers contenant les fichiers .fasta
phage_dir = 'datasets/phage_fragmented/'
refseq_dir = 'datasets/refseq_simulated_metagenome/'

# Liste des fichiers à traiter
fichiers = [
    'Dataset-2_temperate_fragmented.fasta',
    'Dataset-1_virulent_fragmented.fasta',
    'Dataset-2_virulent_fragmented.fasta',
    'Dataset-1_temperate_fragmented.fasta',
    'host_chr_pvog_fragments.fasta'
]

# Chemin du fichier de sortie (dans le dossier datasets)
output_file = 'datasets/fusion_sequences.fasta'

# Créer le dossier 'datasets' si nécessaire
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Fonction pour extraire et modifier l'en-tête
def modifier_header(header):
    # Extraire la première colonne avant le premier "|"
    first_column = header.split('|')[0]
    if first_column.startswith(">Viru"):
        return ">Viru"
    elif first_column.startswith(">Temp"):
        return ">Temp"
    else:
        return ">Bact"

# Fonction pour lire et fusionner les séquences
def fusionner_sequences():
    with open(output_file, 'w') as output:
        # Traitement des fichiers du dossier phage_fragmented
        for filename in fichiers:
            file_path = None
            if filename in os.listdir(phage_dir):
                file_path = os.path.join(phage_dir, filename)
            elif filename in os.listdir(refseq_dir):
                file_path = os.path.join(refseq_dir, filename)

            if file_path:
                with open(file_path, 'r') as file:
                    header = None
                    sequence = []
                    for line in file:
                        line = line.strip()
                        if line.startswith('>'):
                            # Si on a déjà une séquence, on l'écrit dans le fichier
                            if header:
                                output.write(f"{header}\n")
                                output.write(''.join(sequence) + '\n')
                            # Nouveau header
                            header = modifier_header(line)
                            sequence = []
                        else:
                            sequence.append(line)
                    # Ajouter la dernière séquence lue
                    if header:
                        output.write(f"{header}\n")
                        output.write(''.join(sequence) + '\n')

# Exécution de la fusion
fusionner_sequences()
print(f"Les séquences ont été fusionnées et enregistrées dans {output_file}")


Les séquences ont été fusionnées et enregistrées dans datasets/fusion_sequences.fasta


In [3]:
# Fonction pour compter les occurrences de Bact, Temp, et Viru dans le fichier
def compter_occurrences(fichier):
    counts = {'Bact': 0, 'Temp': 0, 'Viru': 0}
    
    with open(fichier, 'r') as file:
        for line in file:
            # Vérifier si la ligne est un en-tête
            if line.startswith('>'):
                # Vérifier le début de l'en-tête
                if line.startswith('>Viru'):
                    counts['Viru'] += 1
                elif line.startswith('>Temp'):
                    counts['Temp'] += 1
                else:
                    counts['Bact'] += 1
    
    return counts

# Appeler la fonction pour compter les occurrences dans le fichier fusionné
resultats = compter_occurrences('datasets/fusion_sequences.fasta')

# Affichage des résultats
print(f"Viru : {resultats['Viru']}")
print(f"Temp : {resultats['Temp']}")
print(f"Bact : {resultats['Bact']}")


Viru : 11565
Temp : 2769
Bact : 104003


In [ ]:
# Fonction pour afficher les 50 premières lignes du fichier fusionné (en-tête + séquences)
def afficher_head(fichier, n_lignes=50):
    with open(fichier, 'r') as file:
        count = 0
        for line in file:
            if count < n_lignes:
                print(line.strip())  # Affiche chaque ligne sans sauter de ligne supplémentaire
                count += 1
            else:
                break

# Appeler la fonction pour afficher les 50 premières lignes du fichier fusionné
afficher_head('datasets/fusion_sequences.fasta', n_lignes=50)


In [5]:
from Bio import SeqIO
import torch
from evo import Evo
from tqdm import tqdm

# Fonction pour lire les séquences du fichier et les diviser en batches
def load_sequences_in_batches(fichier, batch_size=64):
    sequences = []
    
    # Lire le fichier fasta
    for record in SeqIO.parse(fichier, "fasta"):
        sequences.append(str(record.seq))
    
    # Diviser les séquences en batches
    for i in range(0, len(sequences), batch_size):
        yield sequences[i:i + batch_size]

# Créer un modèle Evo
evo_model = Evo('evo-1-131k-base')
model, tokenizer = evo_model.model, evo_model.tokenizer
model.eval()

# S'assurer que tout se fait sur le GPU
device = torch.device("cuda:0")  # Utilisation du GPU 0
model.to(device)  # Déplacer le modèle sur le GPU

# Fonction pour générer les embeddings pour un batch de séquences
def generate_embeddings(sequences, model, tokenizer, device='cuda:0'):
    embeddings = []
    for seq in sequences:
        # Convertir les séquences en indices et les envoyer sur le GPU
        input_ids = torch.tensor(tokenizer.tokenize(seq)).unsqueeze(0).to(device)  # Ajouter la dimension du batch et envoyer sur le GPU

        # Vérifier le type du tensor (affichage du type pour vérification)
        print("Type avant conversion:", input_ids.dtype)

        # Si le type n'est pas long, le convertir en torch.long
        if input_ids.dtype != torch.long:
            input_ids = input_ids.long()

        # Vérifier à nouveau après conversion
        print("Type après conversion:", input_ids.dtype)

        # Passer au modèle pour obtenir les embeddings
        with torch.no_grad():
            outputs = model(input_ids)

        # Ici, `outputs` est un tuple, donc nous accédons au premier élément
        last_hidden_state = outputs[0]  # Récupère la première partie du tuple (les embeddings)
        
        # Moyenne des embeddings pour chaque séquence (ici nous utilisons mean sur les tokens)
        embedding = last_hidden_state.mean(dim=1)
        embeddings.append(embedding)
    
    return embeddings

# Lire les séquences par batches et générer les embeddings
fichier_fasta = 'datasets/fusion_sequences.fasta'
batch_size = 64  # Taille du batch augmentée pour plus de performance

all_embeddings = []  # Liste pour stocker tous les embeddings

# Traitement des séquences par batches
for batch in tqdm(load_sequences_in_batches(fichier_fasta, batch_size=batch_size), desc="Traitement des séquences"):
    # Générer les embeddings pour chaque batch de séquences
    embeddings = generate_embeddings(batch, model, tokenizer, device=device)
    
    # Ajouter les embeddings du batch à la liste
    all_embeddings.extend(embeddings)

# Afficher un exemple des embeddings générés
print(f"Nombre d'embeddings générés : {len(all_embeddings)}")
print("Exemple d'embedding (première séquence) : ", all_embeddings[0])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Traitement des séquences: 0it [00:00, ?it/s]

Type avant conversion: torch.uint8
Type après conversion: torch.int64
Type avant conversion: torch.uint8
Type après conversion: torch.int64
Type avant conversion: torch.uint8
Type après conversion: torch.int64
Type avant conversion: torch.uint8
Type après conversion: torch.int64


Traitement des séquences: 0it [00:13, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.17 GiB. GPU 0 has a total capacity of 21.98 GiB of which 1.89 GiB is free. Including non-PyTorch memory, this process has 20.07 GiB memory in use. Of the allocated memory 16.07 GiB is allocated by PyTorch, and 3.69 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)